In [1]:
import pandas as pd
import requests
import os
from google.cloud import storage
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'

In [3]:
client = storage.Client()
bucket = client.get_bucket('raw-pluto')
bq_client = bigquery.Client()

[API Docs](https://dev.socrata.com/foundry/data.cityofnewyork.us/ic3t-wcy2)  
[OpenData](https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2)  
[Metadata](https://docs.google.com/spreadsheets/d/1TGe-YvR-rqV3sTm2DVl9icoiypo3ug-GDdP1ULdo1_s/edit#gid=169410431)

In [4]:
cols=[
'job__',
'doc__',
'bin__',
'borough',
'block',
'lot',
'latest_action_date',
'pre__filing_date',
'job_type',
'job_status',
'initial_cost',
'building_class',
'existing_occupancy',
'proposed_occupancy',
'existing_dwelling_units',
'proposed_dwelling_units',
'existingno_of_stories',
'proposed_no_of_stories',
'job_status_descrp'
]

col_string=','.join(cols)

In [5]:
JOB_APPLICATIONS='https://data.cityofnewyork.us/resource/ic3t-wcy2.json?$'
LIMIT=10**8
SELECT='*'

In [6]:
get_request=f'{JOB_APPLICATIONS}limit={LIMIT}&$select={col_string}'

d=requests.get(get_request).json()

In [7]:
d2=pd.DataFrame(d)

In [8]:
table_id='pluto-panel.raw_pluto.raw_job_applications'

job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_TRUNCATE'

        )

In [9]:
job = bq_client.load_table_from_dataframe(
    d2, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

In [10]:
table = bq_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 1772967 rows and 19 columns to pluto-panel.raw_pluto.raw_job_applications


## DOB NOW permits
[OpenData](https://data.cityofnewyork.us/Housing-Development/DOB-NOW-Build-Job-Application-Filings/w9ak-ipjd)  
[API Docs](https://dev.socrata.com/foundry/data.cityofnewyork.us/bty7-2jhb)


In [ ]:
d2.yr=d2.latest_action_date.str.split('/', expand = True)[2]

In [ ]:
NOW_APPLICATIONS='https://data.cityofnewyork.us/resource/w9ak-ipjd.json?$'

In [ ]:
resp=requests.get(NOW_APPLICATIONS)
d=resp.json()

In [ ]:
df=pd.DataFrame(d)

In [ ]:
# for c in df.columns:
#     print(c)

In [ ]:
hist_cols=[
    'job_filing_number',
    'filing_status',
    'borough',
    'block',
    'lot',
    'bin',
    'initial_cost',
    'building_type',
    'existing_dwelling_units',
    'proposed_dwelling_units',
    'job_type',
    'existing_stories',
    'proposed_no_of_stories',
    'existing_height',
    'proposed_height',
    'filing_date',
    'current_status_date'
    
]
hist_colstring=','.join(hist_cols)

In [ ]:
get_request=f'{NOW_APPLICATIONS}limit={LIMIT}&$select={hist_colstring}'


now=requests.get(get_request).json()

In [ ]:
df=pd.DataFrame(now)

In [ ]:
table_id='pluto-panel.raw_pluto.raw_job_now_applications'

job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_TRUNCATE'

        )

In [ ]:
job = bq_client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

In [ ]:
table = bq_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)